# French elections 2017 & 2022 
Retrieve and parse oficials XML files of https://www.interieur.gouv.fr and https://www.resultats-elections.interieur.gouv.fr

## Choose election

In [ ]:
ELECTION = 'PR2017_T2'   # PR2022_T1   PR2017_T1  PR2017_T2

In [ ]:
urls = {
    'PR2022_T1' : 'https://www.resultats-elections.interieur.gouv.fr/telechargements/PR2022/resultatsT1/',
    'PR2017_T1' : "https://www.interieur.gouv.fr/avotreservice/elections/telechargements/PR2017/resultatsT1/",
    'PR2017_T2' : "https://www.interieur.gouv.fr/avotreservice/elections/telechargements/PR2017/resultatsT2/"
}
dossier = urls[ELECTION]

In [ ]:
!pip3 install lxml xmltodict
!pip3 install jmespath
import pandas as pd

## Retrieve and parse index.xml file in order to create the file list

In [ ]:
index = pd.read_xml(dossier +'index.xml', xpath='/Election/Departements/Departement', encoding='windows-1252')
index.head(10)

# fix le pb du code region qui n'a pas 3 carac, et fait une liste propres des fichiers qu'il faudra parser
regions = list(index.CodReg3Car.unique())
regions2 = []
for region in regions:
    region2 = str(region)
    if len(region2) <3:
        region2 = '0'+region2
    if len(region2) <3:
        region2 = '0'+region2
    regions2.append(region2)
print(region2)
fix = pd.DataFrame({'CodReg3Car':regions,'CodReg3CarFix':regions2})
fichiers = index.merge(fix, how='left', on='CodReg3Car')[['CodDpt','CodDpt3Car','CodReg3CarFix','LibDpt']]
fichiers.query('CodDpt=="13"')

## Parse files, by departement

In [ ]:
import urllib.request
import xmltodict, jmespath

def treat_dpt(code_region, code_dpt3, code_dpt, libelle_dpt, base_url):
    """Prend en entrées les informations sur le fichier à traiter, et retourne 2 dataframes: 
    - les meta données des communes (participation, nuls, blancs, ...)
    - les résultats des candidats
    """
    if code_dpt == 'ZW':
        # ! walis et futuna ne fonctionne pas
        return None
    
    #corrige les domtom
    if code_dpt in 'ZA,ZB,ZC,ZD,ZS,ZM,ZX'.split(','):
        dpt_pour_insee = '97'
    elif code_dpt == 'ZZ':
        dpt_pour_insee = 'ZZ'
    else:
        dpt_pour_insee = code_dpt
    print(code_dpt, code_dpt3, '==>', dpt_pour_insee)
    
    #créé l'url du fichier, le récupère, et le transforme en dictionnaire
    f = f"{base_url}{code_region}/{code_dpt3}/{code_dpt3}{'' if code_dpt=='ZZ' else 'com'}.xml"
    print(code_region, code_dpt3, code_dpt, libelle_dpt, f)
    urllib.request.urlretrieve(f, "gitignore_fichier.xml")
    xml_data = open('gitignore_fichier.xml', 'r').read()  # Read data
    data = xmltodict.parse(xml_data)  # Parse XML
    
    #liste des communes
    if code_dpt == 'ZZ' :
        print('fr etrangers')
        communes = [jmespath.search('Election.Departement', data)]
        first_commune_path = 'Election.Departement.'
    else:
        communes = jmespath.search('Election.Departement.Communes.Commune[:]', data)
        first_commune_path = 'Election.Departement.Communes.Commune[0:1].'
    
    #prend le bon tour
    tours_tmp = jmespath.search(first_commune_path+'Tours.Tour[-1]', data)
    tour = 'Tours.Tour' if (tours_tmp == None or len(tours_tmp) == 0) else 'Tours.Tour[-1]'
    
    #liste des candidats
    candidats = jmespath.search(first_commune_path + tour + '.Resultats.Candidats.Candidat[:].NomPsn', data) 
    if type(candidats[0]) == list:
        candidats = candidats[0]
    nb_candidats = len(candidats)
    
    #boucle sur les communes pour créer les dataframes de participation (blancs, nuls, ...) et de résultat
    dfs_participations = []
    dfs_resultats = []
    for commune in communes:
        get = lambda r:jmespath.search(r,commune)
        code_commune = get('CodSubCom') 
        
        #fix pour mayotte dans les résulats 2017 ...
        if libelle_dpt == 'Mayotte' and int(code_commune) <600:
            code_commune = str(int(code_commune) + 100) 
        code_insee = dpt_pour_insee if get('CodSubCom') == None else dpt_pour_insee + code_commune

        # ! ignore les données relatives aux arrondissements et aux sections pour ne pas avoir de doublons
        if 'AR' in code_insee or 'SR' in code_insee:
            continue
        
        #données de participation / blanc / nuls 
        meta_communes = pd.DataFrame({"code_dpt":[code_dpt],\
                                    'libelle_dpt':[libelle_dpt],\
                                      'code_insee':[code_insee],\
                         'commune':[get('LibSubCom') or ''],\
                         'votants':[get(tour+'.Mentions.Votants.Nombre')],\
                         'inscrits':[get(tour+'.Mentions.Inscrits.Nombre')],\
                         'nuls':[get(tour+'.Mentions.Nuls.Nombre')], \
                         'blancs':[get(tour+'.Mentions.Blancs.Nombre')],\
                        'exprimes':[get(tour+'.Mentions.Exprimes.Nombre')]}).set_index('code_insee')

        #force les résulats en numeric
        for col in ['votants','inscrits','nuls','blancs','exprimes']:
            meta_communes[col] = pd.to_numeric(meta_communes[col], errors='coerce')
        dfs_participations.append(meta_communes)
        
        #resultat des candidats
        resultats = pd.DataFrame({ 'code_insee':[code_insee]*nb_candidats,
                         'candidat':get(tour+'.Resultats.Candidats.Candidat[:].NomPsn'), 
                         'nb_voix':get(tour+'.Resultats.Candidats.Candidat[:].NbVoix')})
        resultats['nb_voix'] = pd.to_numeric(resultats['nb_voix'], errors='coerce')
        dfs_resultats.append(resultats)
                                   
    communes = pd.concat(dfs_participations)
    resultats = pd.concat(dfs_resultats)
    
    return (communes,resultats)


In [ ]:
# boucle sur les départemennts et aggrege les données
dfs_communes_dpt = []
dfs_resultats_dpt = []
data = {}
def one_file(df):
    #try:
    res = treat_dpt(code_region=df.CodReg3CarFix, code_dpt3=df.CodDpt3Car, code_dpt=df.CodDpt, \
                                 libelle_dpt=df.LibDpt, base_url=dossier)
    if res == None:
        return
    (communes,resultats) = res
    dfs_communes_dpt.append(communes)
    dfs_resultats_dpt.append(resultats)
    #except: 
    #    print('pb avec'+df.CodDpt)
fichiers.apply(one_file, axis=1) #  .query('CodDpt=="ZZ"')

df_communes = pd.concat(dfs_communes_dpt)
df_resultats = pd.concat(dfs_resultats_dpt)

# ! il y a toujours un pb à corriger avec Walis et Futuna

In [ ]:
df_communes.query('commune == "Franconville"')

In [ ]:
candidats = list(df_resultats.candidat.unique())
df_resultats.head(3)

In [ ]:
#vérification qu'il n'y a pas d'erreurs dans les code de commune des fichiers xml (ça arrive...)
df = df_communes.reset_index()
df[df.duplicated(subset='code_insee')]
#df.query('code_insee == "97501"')

In [ ]:
# crée un fichier avec 1 ligne par commune et 1 colonne par candidat
resultats_pivot = df_resultats.pivot(index='code_insee',columns='candidat',values='nb_voix')
out = df_communes.merge(resultats_pivot, left_index=True, right_index=True)
out.head(3)
out.to_csv('gitignore_elections_'+ELECTION+'_brutes.csv')

In [ ]:
#affiche les résultats
conf_agg = {}
for candidat in candidats:
    conf_agg[candidat] = 'sum' 
100.0 * out.agg(conf_agg) / out.exprimes.sum()

In [ ]:
out